# zorang2 Code


In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from emotion import Emotion
import mediapipe as mp
import csv
import glob
import re
from tqdm import tqdm

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-qpzdm8n1 because the default path (/home/sldev1/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
2023-01-13 13:53:56.562533: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sldev1/anaconda3/envs/dms2/lib/python3.7/site-packages/cv2/../../lib64:/usr/local/cuda-10.2/lib64:/usr/local/cuda-10.2/lib64:/usr/local/cuda-10.0/lib64:/usr/local/cuda/extras/CUPTI/lib64
2023-01-13 13:53:56.562554: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
dirpath = "/media/sldev1/T7/"
filepath = ["/media/sldev1/T7/02. 김두용_1205_test.mp4", '/media/sldev1/T7/03. 주용현_1206_test.mp4']
# filepath = glob.glob(dirpath+'*.mp4')
# filepath = sorted(filepath ,key=lambda s: int(re.findall(r'\d+', s)[3]))
# filepath = filepath[:4]
print(filepath)
emo = Emotion()
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

['/media/sldev1/T7/02. 김두용_1205_test.mp4', '/media/sldev1/T7/03. 주용현_1206_test.mp4']


2023-01-13 13:53:57.242723: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2023-01-13 13:53:57.278511: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-13 13:53:57.279336: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.835GHz coreCount: 20 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 298.32GiB/s
2023-01-13 13:53:57.279442: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-13 13:53:57.279807: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 00

In [3]:
for i in tqdm(filepath):
    print(i)

    cap = cv2.VideoCapture(i)
    with mp_face_detection.FaceDetection(
        model_selection=0, min_detection_confidence=0.5) as face_detection:

        with open('/home/sldev1/github/TIL/Internship/DMS2_SMU/FER_Result_SL/'+str(i)[22:-4]+'.csv', 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerow(["frame", "result"])

            while(cap.isOpened()):
                ret, frame = cap.read()
                rgbframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = face_detection.process(rgbframe)
                rgbframe.flags.writeable = True
                rgbframe = cv2.cvtColor(rgbframe, cv2.COLOR_RGB2BGR)

                if results.detections:
                    bbox = results.detections[0].location_data.relative_bounding_box
                    top = rgbframe.shape[0]*bbox.ymin
                    bottom = rgbframe.shape[0]*(bbox.xmin+bbox.height)
                    left = rgbframe.shape[1]*bbox.xmin
                    right = rgbframe.shape[1]*(bbox.xmin+bbox.width)

                    pre_emo, one, two = emo.process(frame[int(left)-10:int(right)+10, int(top)-10:int(bottom)+10])
                    # 0: neutral, 1: joy, 2: sadness, 3: surprise, 4: fear, 5: disgust, 6: anger, 7: contemp
                    cv2.putText(frame, f"Emotion : {pre_emo}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)
                    cv2.putText(frame, f"Frame : {cap.get(1)}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 200), 2)
                    cv2.putText(frame, f"One : {one}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)
                    cv2.putText(frame, f"two : {two}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)


                    cv2.rectangle(frame, (int(left), int(top)), (int(right), int(bottom)), (255, 255, 0), 3)

                cv2.imshow("frame", frame)

                if cv2.waitKey(1) == ord('q'):
                    break

                with open('/home/sldev1/github/TIL/Internship/DMS2_SMU/FER_Result_SL/'+str(i)[22:-4]+'.csv', 'a', newline = '') as file:
                    writer = csv.writer(file)
                    writer.writerow([cap.get(1), pre_emo])
                    file.close()
                
                if cap.get(1) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
                    break
            cap.release()
            cv2.destroyAllWindows()


# 1258 frame에서 멈추는 중

  0%|                                                                                           | 0/2 [00:00<?, ?it/s]

/media/sldev1/T7/02. 김두용_1205_test.mp4


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.5.5) /io/opencv/modules/imgproc/src/resize.cpp:4052: error: (-215:Assertion faile

 50%|█████████████████████████████████████████▌                                         | 1/2 [00:12<00:12, 12.54s/it]

/media/sldev1/T7/03. 주용현_1206_test.mp4


100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:39<00:00, 19.73s/it]
